# Libraries

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

c:\Users\Tautvydas\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


# Functions

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision import transforms
from PIL import Image
import pandas as pd
import numpy as np

# Define the Autoencoder class
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        
        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
        )

        # Flatten and Dense for circle info
        self.flatten = nn.Flatten()
        self.circle_info = nn.Linear(64 * 64 * 64, 3)  # Output: [radii, ccol, crow]

        # Decoder
        self.decoder = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Upsample(scale_factor=2, mode='nearest'),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Upsample(scale_factor=2, mode='nearest'),
            nn.Conv2d(64, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Upsample(scale_factor=2, mode='nearest'),
            nn.Conv2d(32, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Upsample(scale_factor=2, mode='nearest'),
            nn.Conv2d(16, 8, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Upsample(scale_factor=2, mode='nearest'),
            nn.Conv2d(8, 4, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Upsample(scale_factor=2, mode='nearest'),
            nn.Conv2d(4, 1, kernel_size=3, padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        flattened = self.flatten(encoded)
        circle_info_output = self.circle_info(flattened)
        decoded = self.decoder(encoded)
        print("Decoded shape:", decoded.shape)
        return decoded, circle_info_output

# Training Function
def train_autoencoder(model, train_loader, val_loader, epochs=50, lr=1e-3):
    # Define loss functions and optimizer
    image_loss_fn = nn.MSELoss()
    circle_loss_fn = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(epochs):
        model.train()
        train_image_loss = 0
        train_circle_loss = 0

        for images, circles in train_loader:
            images, circles = images.to(device), circles.to(device)
            optimizer.zero_grad()

            decoded, circle_info_output = model(images)
            image_loss = image_loss_fn(decoded, images)
            circle_loss = circle_loss_fn(circle_info_output, circles)
            
            loss = image_loss + circle_loss
            loss.backward()
            optimizer.step()

            train_image_loss += image_loss.item()
            train_circle_loss += circle_loss.item()

        model.eval()
        val_image_loss = 0
        val_circle_loss = 0
        with torch.no_grad():
            for images, circles in val_loader:
                images, circles = images.to(device), circles.to(device)
                decoded, circle_info_output = model(images)
                val_image_loss += image_loss_fn(decoded, images).item()
                val_circle_loss += circle_loss_fn(circle_info_output, circles).item()

        print(f"Epoch {epoch+1}/{epochs} - Train Image Loss: {train_image_loss:.4f}, Train Circle Loss: {train_circle_loss:.4f}, Validation Image Loss: {val_image_loss:.4f}, Validation Circle Loss: {val_circle_loss:.4f}")




# Main

In [3]:
# Data Preparation
df = pd.read_excel('combined_results.xlsx')

image_paths = df['Image Path'].tolist()  
radii = df['circle_radius'].values   
ccol = df['center_y(ccol)'].values
crow = df['center_x(crow)'].values

img_height, img_width = 256, 256  
images = []

transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((img_height, img_width)),
    transforms.ToTensor()
])

for path in image_paths:
    img = Image.open(path)
    img_tensor = transform(img)
    images.append(img_tensor)

images = torch.stack(images)  # Shape: (num_samples, 1, img_height, img_width)
radii = torch.tensor(radii, dtype=torch.float32).view(-1, 1)  # Shape: (num_samples, 1)
ccol = torch.tensor(ccol, dtype=torch.float32).view(-1, 1)
crow = torch.tensor(crow, dtype=torch.float32).view(-1, 1)

circle_params = torch.cat([radii, ccol, crow], dim=1)

# Split data
from sklearn.model_selection import train_test_split
x_train, x_val, circle_train, circle_val = train_test_split(images, circle_params, test_size=0.2, random_state=42)

# Create DataLoaders
train_dataset = TensorDataset(x_train, circle_train)
val_dataset = TensorDataset(x_val, circle_val)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Initialize and Train the Autoencoder
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
model = Autoencoder().to(device)
train_autoencoder(model, train_loader, val_loader)

# Save the model
torch.save(model.state_dict(), 'diffraction_autoencoder_center_merged.pth')

# Testing with validation data
model.eval()
test_images = next(iter(val_loader))[0][:10].to(device)  # Get a batch of 10 images
reconstructed_images, predicted_circles = model(test_images)

print("Predicted Circle Parameters:", predicted_circles.cpu().detach().numpy())

Decoded shape: torch.Size([32, 1, 4096, 4096])


c:\Users\Tautvydas\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\nn\modules\loss.py:610: UserWarning: Using a target size (torch.Size([32, 1, 256, 256])) that is different to the input size (torch.Size([32, 1, 4096, 4096])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: The size of tensor a (4096) must match the size of tensor b (256) at non-singleton dimension 3